In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-29 20:54:54.119363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
sampler = RNNSampler(BASE_TOKENS, 16, 1)

root - DEBUG - 29-Dec-23 21:03:05 - Initial call, object=RNNSampler
root - DEBUG - 29-Dec-23 21:03:05 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.18, 0.17, ..., 0.03, 0.07]]>
root - DEBUG - 29-Dec-23 21:03:05 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.18, 0.17, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 21:03:05 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.20, 0.19, ..., 0.00, 0.00]]>


In [4]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [7]:
sequences, lengths, entropies, log_probs = sampler.sample(3)

root - DEBUG - 29-Dec-23 21:07:11 - Sampling token iteration 0, object=RNNSampler
root - DEBUG - 29-Dec-23 21:07:11 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.18, 0.17, ..., 0.03, 0.07]
 [0.18, 0.17, ..., 0.03, 0.07]
 [0.18, 0.17, ..., 0.03, 0.07]]>
root - DEBUG - 29-Dec-23 21:07:11 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.18, 0.17, ..., 0.00, 0.00]
 [0.18, 0.17, ..., 0.00, 0.00]
 [0.18, 0.17, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 21:07:11 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.20, 0.19, ..., 0.00, 0.00]
 [0.20, 0.19, ..., 0.00, 0.00]
 [0.20, 0.19, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 21:07:11 - Sampling token iteration 1, object=RNNSampler
root - DEBUG - 29-Dec-23 21:07:11 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (3, 10)
[[0.09, 0.13, ..., 0.06, 0.07]
 [0.10, 0.13, ..., 0.06, 0.07]
 [0.08, 0.10, ...

In [8]:
ensemble = ExpressionEnsemble(sequences, lengths)

In [9]:
ensemble

[<Expr: cos(sin(var_x))>,
 <Expr: sin(exp(cos(sin(cos(exp(log(cos(const / const * var_x))))) - var_x)))>,
 <Expr: cos(sin(sin(sin(exp(cos(var_x + const) + const) + const)))) / const>]

In [19]:
sampler = RNNSampler(BASE_TOKENS, 16, 1)
regressor = SymbolicRegressor(sampler, n_samples=16, loss_func=SymbolicLoss())

root - DEBUG - 29-Dec-23 21:16:28 - Initial call, object=RNNSampler
root - DEBUG - 29-Dec-23 21:16:28 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.12, 0.07, ..., 0.07, 0.07]]>
root - DEBUG - 29-Dec-23 21:16:28 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.12, 0.07, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 21:16:28 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (1, 10)
[[0.14, 0.08, ..., 0.00, 0.00]]>


In [20]:
loss_func = SymbolicLoss()
with tf.GradientTape() as tape:
    predictions = regressor({"X": X, "y": y})
    loss = loss_func(y, predictions)
tape.gradient(loss, regressor.sampler.variables)

root - DEBUG - 29-Dec-23 21:16:28 - Sampling token iteration 0, object=RNNSampler
root - DEBUG - 29-Dec-23 21:16:28 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (16, 10)
[[0.12, 0.07, ..., 0.07, 0.07]
 [0.12, 0.07, ..., 0.07, 0.07]
 ...
 [0.12, 0.07, ..., 0.07, 0.07]
 [0.12, 0.07, ..., 0.07, 0.07]]>
root - DEBUG - 29-Dec-23 21:16:28 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (16, 10)
[[0.12, 0.07, ..., 0.00, 0.00]
 [0.12, 0.07, ..., 0.00, 0.00]
 ...
 [0.12, 0.07, ..., 0.00, 0.00]
 [0.12, 0.07, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 21:16:28 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (16, 10)
[[0.14, 0.08, ..., 0.00, 0.00]
 [0.14, 0.08, ..., 0.00, 0.00]
 ...
 [0.14, 0.08, ..., 0.00, 0.00]
 [0.14, 0.08, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 21:16:28 - Sampling token iteration 1, object=RNNSampler
root - DEBUG - 29-Dec-23 21:16:28 - Probs before constraints, object=RNNSampler, func=cal

[<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
 array([[-2.70024687e-02, -9.57684070e-02, -5.94561696e-02,
         -4.21113856e-02, -7.58657455e-02,  5.07981181e-02,
         -2.79207025e-02, -3.35323699e-02,  3.18156555e-02,
         -5.51038384e-02,  3.37079354e-03, -4.71519195e-02,
         -9.85145941e-03, -2.55194306e-02, -8.41552690e-02,
         -2.01842710e-02],
        [-1.13793127e-02, -1.16502911e-01, -2.89877132e-02,
         -2.46814080e-02, -3.45784277e-02,  4.23809029e-02,
         -4.78639454e-02, -6.54723793e-02,  4.40464541e-02,
         -8.79921168e-02,  2.78659239e-02, -7.66438395e-02,
         -1.15299281e-02, -5.34025207e-02, -9.31060463e-02,
         -7.82149509e-02],
        [-7.84863308e-02, -2.18980208e-01, -2.13574231e-01,
          1.14224143e-02, -2.54737914e-01,  3.51732343e-01,
         -2.62287527e-01,  5.55595979e-02, -8.84618238e-03,
         -7.66216666e-02,  2.03775436e-01, -2.73499608e-01,
          3.61622646e-02, -1.92241788e-01, -2.09068149e

In [53]:
regressor.fit(X, y)

root - DEBUG - 29-Dec-23 16:53:46 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (16, 10)
[[0.09, 0.09, ..., 0.08, 0.03]
 [0.09, 0.09, ..., 0.08, 0.03]
 ...
 [0.09, 0.09, ..., 0.08, 0.03]
 [0.09, 0.09, ..., 0.08, 0.03]]>
root - DEBUG - 29-Dec-23 16:53:46 - After constraints, object=RNNSampler, func=call, probs=<Tensor shape: (16, 10)
[[0.09, 0.09, ..., 0.00, 0.00]
 [0.09, 0.09, ..., 0.00, 0.00]
 ...
 [0.09, 0.09, ..., 0.00, 0.00]
 [0.09, 0.09, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 16:53:46 - After normalization, object=RNNSampler, func=call, probs=<Tensor shape: (16, 10)
[[0.10, 0.10, ..., 0.00, 0.00]
 [0.10, 0.10, ..., 0.00, 0.00]
 ...
 [0.10, 0.10, ..., 0.00, 0.00]
 [0.10, 0.10, ..., 0.00, 0.00]]>
root - DEBUG - 29-Dec-23 16:53:46 - Probs before constraints, object=RNNSampler, func=call, probs=<Tensor shape: (16, 10)
[[0.06, 0.11, ..., 0.03, 0.10]
 [0.05, 0.08, ..., 0.06, 0.09]
 ...
 [0.06, 0.12, ..., 0.03, 0.09]
 [0.05, 0.08, ..., 0.05, 0.07]]>


[<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
 array([[ 2.09215909e-01, -9.05192494e-02,  1.86252594e-03,
         -1.70468226e-01, -2.47195572e-01,  3.54040205e-01,
          6.39755577e-02, -6.85974956e-02, -5.34970239e-02,
         -1.18543148e-01, -3.04260015e-01, -1.21034488e-01,
         -9.05701891e-04, -8.59972984e-02, -5.04535884e-02,
         -1.53424218e-01],
        [ 6.30903691e-02, -5.76023348e-02, -1.09945066e-01,
          2.70380024e-02, -3.96955125e-02,  1.19660825e-01,
         -1.04303025e-02,  6.87241256e-02,  2.68642679e-02,
          4.10576584e-04,  8.45538527e-02,  5.11999428e-02,
         -4.02132012e-02, -9.07219872e-02,  1.72602981e-02,
          4.46925834e-02],
        [ 8.22926238e-02, -2.05820091e-02, -7.37485513e-02,
          6.22039801e-03, -2.18808502e-02,  6.82721734e-02,
          2.50284094e-02,  6.81038648e-02,  3.90193239e-03,
          7.59910187e-03,  1.97314862e-02,  6.17177635e-02,
          3.81157850e-03, -7.70138726e-02, -1.44862775e

# 

# TODO constraint avoid x / x

In [ ]:
from scipy.optimize import basinhopping, minimize
from tensorflow.keras.losses import MSE

In [ ]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [ ]:
expression = Expression(tree)

In [ ]:
results = {}
for T in np.logspace(0, 2, 10):
    res = basinhopping(
        lambda constants: MSE(y, expression.eval(X, constants)),
        expression.constants,
        T=T,
    )
    results[T] = res

In [ ]:
[res.x for result in results]

In [ ]:
res = expression.fit(X, y)

In [ ]:
expression.constants = res.x

In [ ]:
go.Figure(
    [go.Scatter(y=expression.eval(X, res.x)), go.Scatter(y=expression.eval(X, [2, 1]))]
)

In [ ]:
model = EvalModel(tree)
model.compile(optimizer=RMSprop(learning_rate=0.05), loss="mse")
model.fit(
    X,
    y,
    batch_size=X.shape[0],
    epochs=1000,
    callbacks=[
        EarlyStopping(monitor="loss", patience=10, start_from_epoch=10, min_delta=0)
    ],
)
model.constants

In [ ]:
def step(tree, X, y, constants, optimizer: Optimizer):
    with tf.GradientTape() as tape:
        y_pred = walk(tree, tf_eval, X=X, constants={"iter": 0, "value": constants})
        loss = MSE(y, y_pred)
    grads = tape.gradient(loss, constants)
    optimizer.apply_gradients([(grads, constants)])
    return grads, loss


def optimize(
    tree, X: pd.DataFrame, y: pd.Series, constants: tf.Variable, max_steps: int = 100
):
    # optimizer = SGD(learning_rate=0.1)
    optimizer = RMSprop(learning_rate=0.1)

    if isinstance(X, pd.DataFrame):
        X = format_as_dict_of_tensor(X)

    if isinstance(y, pd.Series):
        y = tf.convert_to_tensor(y)

    for n in range(max_steps):
        grads, loss = step(tree, X, y, constants, optimizer)
    return constants


def optimize_constants(tree, X: pd.DataFrame, y: pd.Series, max_steps: int = 100):
    num_constants = walk(tree, count_constant)
    constants = tf.Variable(np.random.randn(num_constants), dtype=tf.float32)
    return optimize(tree, X, y, constants, max_steps)

In [ ]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [ ]:
model.fit(X, y)

In [ ]:
xv, yv = np.meshgrid(np.linspace(-4, 4), np.linspace(-4, 4))

In [ ]:
losses = np.empty_like(xv)
grads = np.empty((50, 50, 2))
for i in range(50):
    for j in range(50):
        constants = tf.Variable([xv[i, j], yv[i, j]])
        with tf.GradientTape() as tape:
            y_pred = walk(tree, tf_eval, X=X, constants={"iter": 0, "value": constants})
            loss = MSE(y, y_pred)
        losses[i, j] = loss.numpy()
        grad = tape.gradient(loss, constants)
        grads[i, j] = grad.numpy()

In [ ]:
go.Figure(
    go.Contour(x=np.linspace(-4, 4), y=np.linspace(-4, 4), z=losses, colorscale="RdBu"),
)

In [ ]:
ff.create_quiver(xv, yv, grads[:, :, 0], grads[:, :, 1], scale=0.05, arrow_scale=0.25)

In [ ]:
losses

In [ ]:
grad

In [ ]:
constants = optimize_constants(tree, X, y)

In [ ]:
constants

In [ ]:
go.Figure(
    [
        go.Scatter(
            x=X.squeeze(),
            y=walk(
                tree, tf_eval, X=X, constants={"iter": 0, "value": constants}
            ).numpy(),
        ),
        go.Scatter(x=X.squeeze(), y=y),
    ]
)